# Demo 1: Support Agent with RAG + SQL + Transactions

## Overview

This notebook demonstrates building an AI support agent that:
- Queries **SQL tables** for order data
- Retrieves **user preferences** from KV storage
- Uses **vector RAG** to find relevant policies
- Encodes data in **TOON format** (40-67% token savings)
- Executes **ACID transactions** to update multiple tables atomically

### What You'll Learn

1. How to use ToonDB's SQL interface (`execute_sql`)
2. How to store and retrieve key-value data
3. How to create and query vector collections
4. How to use `ContextQuery` with token budgeting
5. How TOON encoding reduces prompt tokens
6. How to ensure data consistency with ACID transactions

---

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from toondb import Database, ContextQuery, DeduplicationStrategy
from shared.toon_encoder import rows_to_toon
from shared.llm_client import LLMClient, count_tokens
from shared.embeddings import EmbeddingClient

print("✅ All dependencies imported successfully!")

---

## Step 1: Initialize Database with Schema

### 📚 Concept: SQL in ToonDB

ToonDB provides a full SQL interface using `execute_sql()`. You can:
- Create tables with standard DDL
- Insert, update, delete data
- Query with SELECT, JOIN, WHERE, etc.

**Why this matters**: You don't need a separate Postgres instance for structured data.

### How-To: Create Tables

In [ ]:
# Initialize database
db_path = "./support_agent_db"

with Database.open(db_path) as db:
    # Create orders table
    db.execute_sql("""
        CREATE TABLE IF NOT EXISTS orders (
            id INTEGER PRIMARY KEY,
            user_id INTEGER NOT NULL,
            status TEXT NOT NULL,
            eta TEXT,
            destination TEXT NOT NULL,
            total REAL NOT NULL,
            created_at TEXT NOT NULL
        )
    """)
    
    # Create tickets table
    db.execute_sql("""
        CREATE TABLE IF NOT EXISTS tickets (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_id INTEGER NOT NULL,
            reason TEXT NOT NULL,
            created_at TEXT NOT NULL
        )
    """)
    
    # Create audit logs table
    db.execute_sql("""
        CREATE TABLE IF NOT EXISTS audit_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            entity_type TEXT NOT NULL,
            entity_id INTEGER NOT NULL,
            action TEXT NOT NULL,
            details TEXT,
            timestamp TEXT NOT NULL
        )
    """)

print("✅ Created 3 SQL tables: orders, tickets, audit_logs")

### Insert Sample Data

In [ ]:
with Database.open(db_path) as db:
    # Sample orders for user 123
    sample_orders = [
        (1001, 123, "IN_TRANSIT", "2026-01-02", "123 Main St, Seattle, WA", 149.99, "2025-12-28"),
        (1002, 123, "DELIVERED", "2025-12-30", "123 Main St, Seattle, WA", 79.99, "2025-12-20"),
        (1004, 123, "LATE", "2025-12-31", "123 Main St, Seattle, WA", 199.99, "2025-12-22"),
    ]
    
    for order in sample_orders:
        db.execute_sql(f"""
            INSERT OR IGNORE INTO orders VALUES 
            ({order[0]}, {order[1]}, '{order[2]}', '{order[3]}', '{order[4]}', {order[5]}, '{order[6]}')
        """)

print(f"✅ Inserted {len(sample_orders)} orders for user 123")

---

## Step 2: Store User Preferences in KV Storage

### 📚 Concept: Key-Value Storage

ToonDB provides a KV interface with `put()` and `get()`. Use it for:
- User preferences
- Configuration data
- Session state
- Any data accessed by key

**Why this matters**: No need for Redis or separate cache.

### How-To: Store and Retrieve KV Data

In [ ]:
with Database.open(db_path) as db:
    # Store user preferences as KV pairs
    db.put(b"users/123/name", b"Alice Johnson")
    db.put(b"users/123/email", b"alice@example.com")
    db.put(b"users/123/prefs/replacements_over_refunds", b"true")
    db.put(b"users/123/prefs/expedited_shipping_only", b"false")
    
    # Retrieve preferences
    name = db.get(b"users/123/name").decode()
    prefers_replacements = db.get(b"users/123/prefs/replacements_over_refunds").decode()

print(f"✅ Stored preferences for: {name}")
print(f"   Prefers replacements over refunds: {prefers_replacements}")

---

## Step 3: Index Policy Documents with Vector Search

### 📚 Concept: Vector Collections

ToonDB has built-in HNSW vector search. You can:
- Create collections with specified dimensions
- Add documents with embeddings
- Query by semantic similarity

**Why this matters**: No need for Pinecone, Weaviate, or separate vector DB.

### How-To: Create and Populate a Vector Collection

In [ ]:
# Initialize embedding client
embedding_client = EmbeddingClient()
dimension = embedding_client.dimension  # 1536 for text-embedding-3-small

# Sample policy documents
policies = [
    {
        "id": "late_shipment_1",
        "text": "When a customer reports a shipment delay, check if package is still in transit. If yes, offer reroute to alternative address.",
        "source": "late_shipment.txt"
    },
    {
        "id": "late_shipment_2",
        "text": "For orders more than 3 days late, offer expedited replacement shipment. Customer preference should be checked first.",
        "source": "late_shipment.txt"
    },
    {
        "id": "reroute_1",
        "text": "Rerouting is available for packages still in carrier possession. Cannot reroute if already out for delivery.",
        "source": "reroute_guidelines.txt"
    }
]

with Database.open(db_path) as db:
    # Create namespace and collection
    ns = db.namespace("support_system")
    collection = ns.create_collection("policies", dimension=dimension)
    
    # Index each policy
    for policy in policies:
        # Generate embedding
        embedding = embedding_client.embed(policy["text"])
        
        # Add to collection
        collection.add_document(
            id=policy["id"],
            embedding=embedding,
            text=policy["text"],
            metadata={"source": policy["source"]}
        )

print(f"✅ Indexed {len(policies)} policy documents into vector collection")
print(f"   Collection: 'policies' with dimension {dimension}")

---

## Step 4: Query Data with SQL

In [ ]:
user_id = 123

with Database.open(db_path) as db:
    result = db.execute_sql(f"""
        SELECT id, status, eta, destination, total
        FROM orders
        WHERE user_id = {user_id}
        ORDER BY created_at DESC
    """)
    
    orders = result.rows

print(f"📊 Found {len(orders)} orders for user {user_id}:")
for order in orders:
    print(f"   Order #{order['id']}: {order['status']} - ETA: {order['eta']} - ${order['total']}")

---

## Step 5: Encode Results in TOON Format

### 📚 Concept: TOON Encoding

TOON (Tabular Object Oriented Notation) is a compact format for tabular data:

```
table_name[count]{field1,field2,field3}:
value1,value2,value3
value4,value5,value6
```

**Token Savings**: 40-67% fewer tokens than JSON!

### How-To: Convert Rows to TOON

In [ ]:
import json

# Convert to TOON
orders_toon = rows_to_toon(
    "orders",
    orders,
    fields=["id", "status", "eta", "destination", "total"]
)

# Compare with JSON
orders_json = json.dumps(orders, indent=2)

toon_tokens = count_tokens(orders_toon)
json_tokens = count_tokens(orders_json)
savings = json_tokens - toon_tokens
percent = (savings / json_tokens * 100) if json_tokens > 0 else 0

print("📦 TOON Format:")
print(orders_toon)
print(f"\n💾 Token Comparison:")
print(f"   JSON:  {json_tokens} tokens")
print(f"   TOON:  {toon_tokens} tokens")
print(f"   Saved: {savings} tokens ({percent:.1f}% reduction)")

---

## Step 6: Retrieve Relevant Policies with ContextQuery

### 📚 Concept: ContextQuery with Token Budgeting

`ContextQuery` is ToonDB's powerful retrieval interface that:
- Combines vector + keyword search (hybrid with RRF)
- Enforces strict token budgets (prevents prompt overflow)
- Deduplicates results semantically

**Why this matters**: Ensures your context always fits in the prompt.

### How-To: Build a Context Query

In [ ]:
user_question = "My order is late. I'm traveling tomorrow. Can you reroute or replace it?"

# Generate query embedding
query_embedding = embedding_client.embed(user_question)

with Database.open(db_path) as db:
    ns = db.namespace("support_system")
    collection = ns.collection("policies")
    
    # Build context query with token budget
    ctx = (
        ContextQuery(collection)
        .add_vector_query(query_embedding, weight=0.7)  # Semantic similarity
        .add_keyword_query("late shipment reroute replacement", weight=0.3)  # Keyword match
        .with_token_budget(1200)  # Hard limit: never exceed this
        .with_deduplication(DeduplicationStrategy.SEMANTIC)  # Remove similar chunks
        .execute()
    )

print(f"🔍 Retrieved {len(ctx.documents)} relevant policy chunks")
print(f"   Token budget: ~{ctx.total_tokens} / 1200")
print(f"\n📄 Relevant policies:")
for i, doc in enumerate(ctx.documents, 1):
    print(f"   {i}. {doc.text[:100]}...")

---

## Step 7: Generate LLM Response

In [ ]:
llm = LLMClient()

system_message = """You are a customer support agent. Follow company policies strictly.
Do not invent shipment facts. Base your response on the order data and policies provided."""

prompt = f"""User: Alice Johnson (ID: 123)
Preferences: replacements_over_refunds=true

Question: {user_question}

Recent orders (TOON format):
{orders_toon}

Relevant policies:
{ctx.as_markdown()}

Provide a helpful response and suggest specific actions.
"""

response = llm.complete(prompt, system_message=system_message)

print("🤖 Agent Response:")
print("=" * 70)
print(response)
print("=" * 70)

---

## Step 8: Execute ACID Transaction

### 📚 Concept: ACID Transactions

ToonDB provides ACID guarantees (Atomicity, Consistency, Isolation, Durability):
- All operations succeed or all fail (no partial updates)
- Uses MVCC (Multi-Version Concurrency Control)
- WAL (Write-Ahead Logging) for durability
- SSI (Serializable Snapshot Isolation)

**Why this matters**: Critical data updates are always consistent.

### How-To: Atomic Multi-Table Update

In [ ]:
with Database.open(db_path) as db:
    timestamp = datetime.now().isoformat()
    order_id = 1004  # The late order
    
    # All 3 operations are atomic - all succeed or all fail
    
    # 1. Update order status
    db.execute_sql(f"""
        UPDATE orders
        SET status = 'REROUTE_REQUESTED'
        WHERE id = {order_id}
    """)
    
    # 2. Create support ticket
    db.execute_sql(f"""
        INSERT INTO tickets (order_id, reason, created_at)
        VALUES ({order_id}, 'Reroute request - customer traveling', '{timestamp}')
    """)
    
    # 3. Log audit trail
    db.execute_sql(f"""
        INSERT INTO audit_logs (entity_type, entity_id, action, details, timestamp)
        VALUES ('order', {order_id}, 'reroute_requested', 'User requested reroute due to travel', '{timestamp}')
    """)

print("✅ ACID Transaction Complete!")
print(f"   Updated order #{order_id}")
print(f"   Created support ticket")
print(f"   Logged audit trail")
print(f"\n💡 All 3 operations committed atomically - guaranteed consistent!")

---

## Verify Transaction Results

In [ ]:
with Database.open(db_path) as db:
    # Check order status
    order = db.execute_sql(f"SELECT * FROM orders WHERE id = 1004").rows[0]
    print(f"📦 Order #1004 status: {order['status']}")
    
    # Check ticket created
    tickets = db.execute_sql(f"SELECT * FROM tickets WHERE order_id = 1004").rows
    print(f"🎫 Tickets created: {len(tickets)}")
    
    # Check audit log
    audits = db.execute_sql(f"SELECT * FROM audit_logs WHERE entity_id = 1004").rows
    print(f"📝 Audit entries: {len(audits)}")
    for audit in audits:
        print(f"   - {audit['action']}: {audit['details']}")

---

## Summary: What We Accomplished

### ✅ Features Demonstrated

1. **SQL Interface** - Created tables, inserted data, queried with WHERE/ORDER BY
2. **KV Storage** - Stored and retrieved user preferences
3. **Vector Search** - Indexed policies, queried by semantic similarity
4. **TOON Encoding** - Reduced tokens by ~50-70% vs JSON
5. **ContextQuery** - Combined vector + keyword search with token budget
6. **ACID Transactions** - Atomically updated 3 tables

### 💡 Key Insights

**No Glue Code Needed**
- Traditional: Postgres + Redis + Pinecone + ETL scripts
- ToonDB: All in one database with consistent API

**Token Efficiency**
- TOON format saves 40-67% tokens
- More context in prompts = better AI responses
- Lower API costs

**Production-Ready**
- ACID guarantees data consistency
- Token budgets prevent prompt overflow
- HNSW for fast vector search

### 🚀 Next Steps

Try modifying:
- Add more policy documents and see how retrieval adapts
- Change token budget and observe context trimming
- Add more tables and create complex joins
- Experiment with different embedding models

---

## Resources

- [ToonDB Documentation](https://github.com/toondb/toondb)
- [Python SDK](https://github.com/toondb/toondb-python-sdk)
- [Demo Source Code](../1_support_agent/)
